In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4177/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_6_7_7,0.923488,0.845777,0.854510,0.862057,9.227380,12.420743,6.698016,9.727782,"Hidden Size=[30, 25], regularizer=0.1, learnin..."
1,model_6_7_6,0.922506,0.852655,0.862966,0.868813,9.345759,11.866856,6.308726,9.251298,"Hidden Size=[30, 25], regularizer=0.1, learnin..."
19,model_6_6_4,0.885488,0.974398,0.869271,0.934299,13.810178,3.851471,14.172795,8.708566,"Hidden Size=[30, 25], regularizer=0.1, learnin..."
20,model_6_6_5,0.885086,0.967873,0.847652,0.922057,13.858716,4.833076,16.516573,10.331194,"Hidden Size=[30, 25], regularizer=0.1, learnin..."
21,model_6_6_3,0.883743,0.979190,0.878829,0.940858,14.020656,3.130548,13.136536,7.839250,"Hidden Size=[30, 25], regularizer=0.1, learnin..."
22,model_6_6_6,0.882325,0.960847,0.809413,0.903117,14.191664,5.890069,20.662258,12.841700,"Hidden Size=[30, 25], regularizer=0.1, learnin..."
23,model_6_6_2,0.880567,0.982257,0.881999,0.943920,14.403646,2.669170,12.792928,7.433290,"Hidden Size=[30, 25], regularizer=0.1, learnin..."
24,model_3_6_2,0.879500,0.724721,0.880286,0.832054,14.532381,19.885885,21.307829,20.555058,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
25,model_3_6_1,0.877448,0.721336,0.883320,0.833073,14.779754,20.130432,20.767746,20.430338,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
26,model_6_6_1,0.877041,0.983550,0.884735,0.945750,14.828943,2.474709,12.496285,7.190749,"Hidden Size=[30, 25], regularizer=0.1, learnin..."
